In [1]:
#import Modul Python yang akan digunakan

import time
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
#transformasi data

trans_train = transforms.Compose([transforms.Lambda(lambda image: image.convert('RGB')),
                                  transforms.Resize((64,64)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5,), (1.0,))])

trans = transforms.Compose([transforms.Lambda(lambda image: image.convert('RGB')),
                            transforms.Resize((64,64)),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,), (1.0,))])

train_set = dset.MNIST(root='./data', train=True, transform=trans_train, download=True)
test_set = dset.MNIST(root='./data', train=False, transform=trans)
batch_size = 64

In [3]:
#cek data

print(train_set)
image,label=train_set[10]
print(image.shape)
print(label)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               Lambda()
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5,), std=(1.0,))
           )
torch.Size([3, 64, 64])
3


In [4]:
#loader data untuk training dan testing

train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,shuffle=False)

In [5]:
#inisialisasi Model AlexNet
Fitur = 10

model1 = torchvision.models.alexnet(weights='AlexNet_Weights.DEFAULT')
model1.classifier.add_module('6', nn.Dropout(p=0.5, inplace=False))
model1.classifier.add_module('7', nn.Linear(in_features=4096, out_features=Fitur))
model1.classifier.add_module('8', nn.Softmax(dim=1))
model = model1
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
def train(epoch):
    model.train()
    start_time = time.time()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # if torch.cpu.is_available():
        #     data, target = data.cpu(), target.cpu()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()
        
        # Menghitung jumlah prediksi yang benar
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        if batch_idx % 100 == 0:
            print('\rEpoch: {} {:.0f}%\t     Loss: {:.6f}'.format(
                epoch,
                100. * batch_idx / len(train_loader), loss.item()), end='')
    end_time = time.time()
    print("\nLama waktu Training pada epoch {}: {} Menit".format(epoch, ((end_time - start_time)/60)))
    
    # Menghitung dan mencetak akurasi pelatihan
    train_accuracy = 100. * correct / len(train_loader.dataset)
    train_akurasi.append(train_accuracy)
    print('Akurasi pada Training pada epoch {}: {:.2f}%'.format(epoch, train_accuracy))

In [7]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        # if torch.cpu.is_available():
        #     data, target = data.cpu(), target.cpu()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, reduction='sum').item()
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    
    
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    acc=100. * float(correct.to(torch.device('cpu')).numpy())
    print('\nHasil Testing: Rata-Rata loss: {:.4f}, Akurasi: {:.4f}%\n'.format(
        test_loss, acc / len(test_loader.dataset)))
    
    test_accuracy.append(acc / len(test_loader.dataset))

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01)


train_losses = []
test_losses =[]
test_accuracy = []
train_akurasi = []

for epoch in range(1, 2):
    train(epoch)
    test()
    

Epoch: 1 96%	     Loss: 1.474141
Lama waktu Training pada epoch 1: 7.27546318769455 Menit
Akurasi pada Training pada epoch 1: 90.81%

Hasil Testing: Rata-Rata loss: 1.4828, Akurasi: 97.9300%



In [9]:
# membuat prediksi dan ground truth labels
y_pred = []
y_true = []
for data, target in test_loader:
    data, target = Variable(data), Variable(target)
    output = model(data)
    pred = output.data.max(1, keepdim=True)[1].cpu().numpy().squeeze()
    y_pred.extend(pred)
    y_true.extend(target.data.cpu().numpy())

# membuat confusion matrix dan classification report
conf_mat = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred)

print('Confusion Matrix:')
print(conf_mat)

Confusion Matrix:
[[ 965    0    3    0    0    1   10    1    0    0]
 [   0 1125    3    2    0    0    2    3    0    0]
 [   1    2 1022    2    0    0    1    3    1    0]
 [   0    0    3  996    0    5    0    2    4    0]
 [   0    2    3    0  933    0    4    6    2   32]
 [   0    1    1   13    0  867    4    2    4    0]
 [   1    0    0    1    0    3  952    0    1    0]
 [   0    1    8    1    0    0    0 1009    2    7]
 [   2    1    8    5    2    1    3    2  937   13]
 [   1    1    2    2    2    3    0    4    7  987]]


In [10]:
print('Classification Report:')
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.97      0.99      0.98      1032
           3       0.97      0.99      0.98      1010
           4       1.00      0.95      0.97       982
           5       0.99      0.97      0.98       892
           6       0.98      0.99      0.98       958
           7       0.98      0.98      0.98      1028
           8       0.98      0.96      0.97       974
           9       0.95      0.98      0.96      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000

